In [1]:
#Import libraries

import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam


In [2]:
#Load the dataset and preprocess it

titanic = pd.read_csv("./Titanic data.csv")
titanic.drop(columns=["Cabin"], inplace=True)  # Remove the 'Cabin' column as it has many missing values
titanic["Age"].fillna(titanic["Age"].mean(), inplace=True)  # Fill missing 'Age' values with the mean
titanic["Embarked"].fillna(titanic["Embarked"].mode()[0], inplace=True)  # Fill missing 'Embarked' values with the mode


In [3]:
#Prepare the data for the model

X = titanic.drop(columns=["Survived", "PassengerId", "Name", "Ticket"])  # Features
y = titanic["Survived"]  # Target variable

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

numeric_features = ["Age", "Fare"]
numeric_transformer = StandardScaler()

categorical_features = ["Pclass", "Sex", "Embarked"]
categorical_transformer = OneHotEncoder()

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ])

X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)


In [4]:
# Define the neural network model

model = Sequential()
model.add(Dense(64, activation="relu", input_shape=(X_train.shape[1],)))
model.add(Dense(32, activation="relu"))
model.add(Dense(1, activation="sigmoid"))

model.compile(optimizer=Adam(learning_rate=0.001), loss="binary_crossentropy", metrics=["accuracy"])

#Train the model

history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2, verbose=1)


Epoch 1/100
18/18 [==============================] - 2s 29ms/step - loss: 0.6444 - accuracy: 0.6591 - val_loss: 0.5845 - val_accuracy: 0.7552
Epoch 2/100
18/18 [==============================] - 0s 7ms/step - loss: 0.5662 - accuracy: 0.7417 - val_loss: 0.5121 - val_accuracy: 0.8462
Epoch 3/100
18/18 [==============================] - 0s 7ms/step - loss: 0.5116 - accuracy: 0.7838 - val_loss: 0.4590 - val_accuracy: 0.8252
Epoch 4/100
18/18 [==============================] - 0s 7ms/step - loss: 0.4791 - accuracy: 0.7786 - val_loss: 0.4253 - val_accuracy: 0.8252
Epoch 5/100
18/18 [==============================] - 0s 8ms/step - loss: 0.4605 - accuracy: 0.7873 - val_loss: 0.4094 - val_accuracy: 0.8252
Epoch 6/100
18/18 [==============================] - 0s 8ms/step - loss: 0.4488 - accuracy: 0.7856 - val_loss: 0.4054 - val_accuracy: 0.8182
Epoch 7/100
18/18 [==============================] - 0s 7ms/step - loss: 0.4462 - accuracy: 0.8014 - val_loss: 0.3946 - val_accuracy: 0.8462
Epoch 8/100


Epoch 59/100
18/18 [==============================] - 0s 7ms/step - loss: 0.3748 - accuracy: 0.8295 - val_loss: 0.3979 - val_accuracy: 0.8182
Epoch 60/100
18/18 [==============================] - 0s 7ms/step - loss: 0.3744 - accuracy: 0.8313 - val_loss: 0.3895 - val_accuracy: 0.8462
Epoch 61/100
18/18 [==============================] - 0s 7ms/step - loss: 0.3737 - accuracy: 0.8278 - val_loss: 0.3907 - val_accuracy: 0.8252
Epoch 62/100
18/18 [==============================] - 0s 8ms/step - loss: 0.3714 - accuracy: 0.8295 - val_loss: 0.3952 - val_accuracy: 0.8531
Epoch 63/100
18/18 [==============================] - 0s 7ms/step - loss: 0.3739 - accuracy: 0.8330 - val_loss: 0.3912 - val_accuracy: 0.8531
Epoch 64/100
18/18 [==============================] - 0s 7ms/step - loss: 0.3702 - accuracy: 0.8295 - val_loss: 0.3927 - val_accuracy: 0.8531
Epoch 65/100
18/18 [==============================] - 0s 8ms/step - loss: 0.3713 - accuracy: 0.8330 - val_loss: 0.3987 - val_accuracy: 0.8252
Epoch 

In [5]:
#Evaluate the model on train and test sets

train_loss, train_accuracy = model.evaluate(X_train, y_train, verbose=0)
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)

print(f"Train accuracy: {train_accuracy:.2f}")
print(f"Test accuracy: {test_accuracy:.2f}")


Train accuracy: 0.84
Test accuracy: 0.82


In [6]:
#Make predictions on the test set

predictions = model.predict(X_test)
threshold = 0.5
binary_predictions = (predictions >= threshold).astype(int)
print("Predictions:")
print(binary_predictions.squeeze())


6/6 [==============================] - 0s 3ms/step
Predictions:
[0 0 0 1 1 1 1 0 1 1 0 0 0 0 0 1 0 1 0 0 0 0 1 0 0 0 0 0 0 1 0 1 0 1 0 0 1
 1 1 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 1 0 1 1 1 0 1 1 0 0 1 0 0 1 1 1 1 1 1
 0 0 1 1 1 0 0 1 1 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 1
 0 1 0 0 0 0 0 1 0 0 1 1 1 0 0 1 1 1 0 1 0 0 1 0 1 1 0 0 0 0 1 0 0 1 1 0 0
 1 0 0 0 0 1 0 0 0 1 1 1 0 0 0 1 0 0 0 1 0 0 1 1 0 1 0 0 0 1 1]
